In [1]:
import pandas as pd
with open('./counting_data/지속가능경영보고서_키워드.csv','r',encoding='utf-8') as f:
    keywords = [lines.strip() for lines in f.readlines()][1:]
keyword_label = [tuple(keyword.split(',')) for keyword in keywords]
df_news = pd.read_csv('./counting_data/total_kosdaq_data.csv')

In [4]:
import datetime
current_datetime = datetime.datetime.now()
now_year = current_datetime.year
print("현재년도:", now_year)

현재년도: 2024


In [6]:
from counting_data.mariadb import MariaDB
with open('./counting_data/aws_config','r',encoding='utf-8') as f:
    db_config = {lines.split('=')[0].strip():lines.split('=')[1].strip() for lines in f.readlines()}
db = MariaDB(db_config)

In [9]:
df_year = df_news[df_news['Date'].apply(lambda x: x.split('.')[0] == str(2023))] # 현재날짜 년도의 컨텐츠만

In [15]:
len(df_year) , len(df_news)
df_year

,Unnamed: 0,Company,Date,URL,Sentence
3228897,3616483,엔젤로보틱스,2023.01.09,https://n.news.naver.com/mnews/article/023/000...,코로나 팬데믹대유행 이후 3년 만에 전면 오프라인으로 열린 세계 최대 IT 전시회 ...
3228898,3616484,엔젤로보틱스,2023.01.09,https://n.news.naver.com/mnews/article/023/000...,부스마다 길게 늘어선 줄과 마스크를 쓰지 않은 참가자들의 모습은 코로나 이전으로 완...
3228899,3616485,엔젤로보틱스,2023.01.09,https://n.news.naver.com/mnews/article/023/000...,7일 CES 전시가 한창인 미국 라스베이거스 유레카파크에서 선정윤 서울대 재료공학부...
3228900,3616486,엔젤로보틱스,2023.01.09,https://n.news.naver.com/mnews/article/023/000...,이들은 규모는 코로나 이전을 회복했지만 기업들이 추구하는 전시의 방향성이 완전히 달...
3228901,3616487,엔젤로보틱스,2023.01.09,https://n.news.naver.com/mnews/article/023/000...,공경철 교수는 이전에 없었던 놀라운 기술은 거의 등장하지 않았지만 기존에 기업들이 ...
...,...,...,...,...,...
4268944,4738482,리더스 기술투자,2023.12.19,https://n.news.naver.com/mnews/article/016/000...,승계형 그룹 상장사213개의 시가총액은 2020년 12월 30일 1309조3780억...
4268945,4738483,리더스 기술투자,2023.12.19,https://n.news.naver.com/mnews/article/016/000...,반면 같은 기간 창업형 그룹들의 상장사58개의 시가총액은 223조9650억원에서 2...
4268946,4738484,리더스 기술투자,2023.12.19,https://n.news.naver.com/mnews/article/016/000...,리더스인덱스는 이번 조사에 대해 팬데믹 종료와 함께 정보기술IT 플랫폼 게임 등 창...
4268947,4738485,리더스 기술투자,2023.12.19,https://n.news.naver.com/mnews/article/016/000...,2020년부터 올해까지 시가총액이 가장 많이 증가한 그룹은 LG47조828억원였고 ...


In [16]:
keyword_counts = {keyword: {'E': 0, 'S': 0, 'G': 0} for keyword, _ in keyword_label}

for index, row in df_year.iterrows():
    sentence = row['Sentence']
    for keyword, label in keyword_label:
        count = sentence.count(keyword)
        if count > 0:
            keyword_counts[keyword][label.strip()] += count

In [22]:
key = keyword_counts.keys()
E = [keyword_counts[key]['E'] for key in keyword_counts]
S = [keyword_counts[key]['S'] for key in keyword_counts]
G = [keyword_counts[key]['G'] for key in keyword_counts]
data = list(zip(key,E,S,G))

In [24]:
db.insert_many('tb_ESG_keyword','Keyword,E_Count,S_Count,G_Count',data)

True